# Loading InstrumentSDK package

In [1]:
import InstrumentSDK.APT as APT

Exception: Your operating system is not supported. Thorlabs' APT API only works on Windows.

# Controlling Elliptec motors

In [9]:
ports = ELL.find_ports()
print(ports)
motor = ELL.Motor(ports[0].device)

Connection established!


In [10]:
motor.move_absolute(50,'0')

Sending SET command: b'0ma00004DBC'
Status/Response may be incomplete!
Returned status: None
Status is None


# Controlling APT motor (goniometer)

In [1]:
import InstrumentSDK.APT as APT
print( APT.list_available_devices() )
unit = 100/540
motor = APT.Motor(40219394)
motor.set_move_home_parameters(ZeroOffset=26)
motor.move_home()

Exception: Your operating system is not supported. Thorlabs' APT API only works on Windows.

In [ ]:
motor.move_by(90*unit)